In [ ]:
!pip install binance-connector ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 2.2 MB/s 
     |████████████████████████████████| 435 kB 11.2 MB/s 
     |████████████████████████████████| 3.1 MB 40.2 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 4.0 MB 33.9 MB/s 
     |████████████████████████████████| 74 kB 1.9 MB/s 
     |████████████████████████████████| 251 kB 74.0 MB/s 
  Created wheel for autobahn: filename=autobahn-22.5.1-cp37-cp37m-linux_x86_64.whl size=635258 sha256=69031f2754e3d6ecf74a9472a3591afac324b25cda200c90d96435779f1e0c9a
  Stored in directory: /root/.cache/pip/wheels/41/3d/10/4af119dc09a42cb9bb3bb96cdfc0af08ede92dabafe366afbb
  Created wheel for ta: filename=ta-0.10.1-py3-none-any.whl size=28985 sha256=b1513a0eefb6036b147baed388efa1a92419ec89925e050406564d771059986d
  Stored in directory: /root

In [ ]:

# Version 2.0 - download from first to last
#  !pip install binance-connector ta
from typing import *
from binance.spot import Spot as Client
import datetime as datetime
import pandas as pd
import time
import os

def ms_to_dt(ms: int) -> datetime.datetime:
    return datetime.datetime.utcfromtimestamp(ms / 1000)

client = Client()
server_time = client.time()
print(ms_to_dt(server_time['serverTime']))
end_time = server_time['serverTime'] - 61000
print(ms_to_dt(end_time))


def data_tocsv(candles, my_symbol):
    path = 'binance_csvs2/'
    df = pd.DataFrame(candles)
    df.to_csv(path+my_symbol+'.csv', mode = 'a', header = False, index=False)


def read_csv(my_symbol):
    path = 'binance_csvs2/'
    dt = pd.read_csv(path+my_symbol+'.csv', names = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l'])
    dt['z'] = pd.to_datetime(dt['a'], unit='ms')
    # dt = dt.set_index('a')
    # print("Reading db...")
    # print(dt.tail(5))
    return dt


def get_first_last_timestamp(my_symbol: str) -> Union[Tuple[None, None], Tuple[float, float]]:
    print("Scanning Timestamp of older Data")
    existing_data = read_csv(my_symbol)
    
    if len(existing_data) == 0:
        return None, None

    first_ts = min(existing_data['a'])
    last_ts = max(existing_data['a'])
    print("LAST_TS in db found", ms_to_dt(last_ts))
    
    return first_ts, last_ts

def collect_all(my_symbol):
    client = Client()
    path = 'binance_csvs2/'
    if os.path.isfile(path+my_symbol+'.csv'):
        oldest_ts, most_recent_ts = get_first_last_timestamp(my_symbol)
        next_todownload_ts = most_recent_ts + 1
    else:
        # Initial Request
        os.makedirs(path)
        kline_data = client.klines(symbol=my_symbol, interval='1m', startTime=0,  endTime=end_time, limit=1000)
        print(my_symbol, ": Collected", len(kline_data),"older data from",  
                    ms_to_dt(kline_data[0][0]), "to" , ms_to_dt(kline_data[-1][0]))
        data_tocsv(kline_data, my_symbol)
        # oldest_ts = kline_data[0][0]
        most_recent_ts = kline_data[-1][0]
        next_todownload_ts = most_recent_ts + 1
        
    data_to_insert = []
    
    while True:
            kline_data = client.klines(symbol=my_symbol, interval='1m', startTime=next_todownload_ts, endTime=end_time, limit=1000)

            if kline_data is None:
                time.sleep(3)  # Pause in case an error occurs during the request
                continue

            if len(kline_data) == 0:
                print(my_symbol, ": Stopped. No more available latest kline data from ",
                            ms_to_dt(most_recent_ts))
                break

            most_recent_ts = kline_data[-1][0]
            next_todownload_ts = most_recent_ts + 1

            print(my_symbol, ": Collected", len(kline_data),"older data from",  
                        ms_to_dt(kline_data[0][0]), "to" , ms_to_dt(most_recent_ts))
        

            data_to_insert = data_to_insert + kline_data

            if len(data_to_insert) >= 50000:
                data_tocsv(data_to_insert, my_symbol)
                print("Saving older Data to DISK")
                print(len(data_to_insert))
                data_to_insert.clear()
      
    data_tocsv(data_to_insert, my_symbol)
    print("Saving Data to DISK")
    print(len(data_to_insert))
    data_to_insert.clear()


relevant = ['BTCUSDT']

for list in relevant:
    print("Downloading", list, "data")
    collect_all(list)

# [
#   [
#     1499040000000,      // Open time
#     "0.01634790",       // Open
#     "0.80000000",       // High
#     "0.01575800",       // Low
#     "0.01577100",       // Close
#     "148976.11427815",  // Volume
#     1499644799999,      // Close time
#     "2434.19055334",    // Quote asset volume
#     308,                // Number of trades
#     "1756.87402397",    // Taker buy base asset volume
#     "28.46694368",      // Taker buy quote asset volume
#     "17928899.62484339" // Ignore.
#   ]
# ]


ModuleNotFoundError: ignored

In [ ]:
import numpy as np
import ta

path = 'binance_csvs2/'
my_symbol = 'BTCUSDT'
dt = pd.read_csv(path+my_symbol+'.csv', names = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l'])
columns = ['a', 'b', 'c', 'd', 'e', 'f', 'h', 'i']
df1 = pd.DataFrame(dt, columns=columns)
df1.columns = ['date_time', 'open', 'high', 'low', 'close', 'volume', 'vol_price', 'trades']
df1['date_time'] = pd.to_datetime(df1['date_time'], unit='ms')
df1.set_index('date_time', drop=True, inplace=True)

def checkcross(dframe):
  series = dframe['strsi_k'] > dframe['strsi_d']
  return series.diff()


# df1['strsi_k'] = ta.momentum.stochrsi_k(df1.close)
# df1['strsi_d'] = ta.momentum.stochrsi_d(df1.close)
# for i in (8,14,50):
#   df1['EMA_'+str(i)] = ta.trend.ema_indicator(df1.close, window=i)
# df1['atr'] = ta.volatility.average_true_range(df1.high, df1.low, df1.close)
# df1.dropna(inplace=True)
# df1['cross'] = checkcross(df1)
# df1['TP'] = df1.close + (df1.atr * 5)
# df1['SL'] = df1.close - (df1.atr * 3)
# df1['Buysignal'] = np.where((df1.cross) & (df1.close > df1.EMA_8) & (df1.EMA_8 > df1.EMA_14) & (df1.EMA_14 > df1.EMA_50) ,1 , 0)

# resampling tha data function

TF_EQUIV = {"1m": "1Min", "5m": "5Min", "15m": "15Min", "30m": "30Min", "1h": "1H", "4h": "4H", "12h": "12H", "1d": "D"}

def resample_timeframe(data: pd.DataFrame, tf: str) -> pd.DataFrame:
    return data.resample(TF_EQUIV[tf]).agg(
        {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum" , "vol_price": "sum", "trades": "sum"})




In [ ]:


df5 = resample_timeframe(df1,'5m')

df5['strsi_k'] = ta.momentum.stochrsi_k(df5.close)
df5['strsi_d'] = ta.momentum.stochrsi_d(df5.close)

for i in (8,14,50):
  df5['EMA_'+str(i)] = ta.trend.ema_indicator(df5.close, window=i)

df5['atr'] = ta.volatility.average_true_range(df5.high, df5.low, df5.close)
df5.dropna(inplace=True)

df5['cross'] = checkcross(df5)
df5['TP'] = df5.close + (df5.atr * 2)
df5['SL'] = df5.close - (df5.atr * 1)
df5['Buysignal'] = np.where((df5.cross) & (df5.close > df5.EMA_8) & (df5.EMA_8 > df5.EMA_14) & (df5.EMA_14 > df5.EMA_50) ,1 , 0)

# Get sell dates
selldates = []
outcome = []

for i in range(len(df5)):
  if df5.Buysignal.iloc[i] == 1 :
    k = 1
    SL = df5.SL.iloc[i]
    TP = df5.TP.iloc[i]
    print(SL, ' ' ,TP)
    in_position = True
    while in_position and  i+k < len(df5):
      print(i+k)
      looping_close = df5.close.iloc[i+k]
      print(looping_close)
      if looping_close >= TP:
        selldates.append(df5.iloc[i+k].name)
        outcome.append('TP')
        in_position = False
      elif looping_close <= SL:
        selldates.append(df5.iloc[i+k].name)
        outcome.append('SL')
        in_position = False
      k +=1

df5.loc[selldates, 'Sellsignal']  = 1
df5.loc[selldates, 'Outcome'] = outcome
df5.Sellsignal = df5.Sellsignal.fillna(0).astype(int)

In [ ]:
df5 = df5[(df5.Buysignal == 1) | (df5.Sellsignal == 1)]
df5 = df5[(df5.Buysignal.diff() == 1) | (df5.Sellsignal.diff() == 1)]

In [ ]:
df5.Outcome.value_counts()